In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tables
import numpy as np
import os
from itertools import repeat
import random
import math
from tqdm import tqdm_notebook as tqdm
tqdm().pandas(desc="")
import sys, gc
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import model_tuner
import scoring, utils

## Imputation

In [ ]:
MATCHEDHIT_MISSING_COL = ['MatchedHit_X[2]', 'MatchedHit_X[3]', 'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 
                           'MatchedHit_Z[2]', 'MatchedHit_Z[3]', 'MatchedHit_DX[2]', 'MatchedHit_DX[3]', 
                           'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]']
CLOSESTHIT_MISSING_COL = ['closest_x_per_station[2]', 'closest_x_per_station[3]', 
                    'closest_y_per_station[2]', 'closest_y_per_station[3]',
                    'closest_T_per_station[2]', 'closest_T_per_station[3]',
                    'closest_z_per_station[2]', 'closest_z_per_station[3]',
                    'closest_dx_per_station[2]', 'closest_dx_per_station[3]',
                    'closest_dy_per_station[2]', 'closest_dy_per_station[3]']

In [ ]:
def missing_value_imputation(dataDF, essential=False, ratio_f=True, substitution_f=True, mean_f=True, angle_f=True, average_xy_f=True):
    imputation_dict = {
        'MatchedHit_X[2]':'Lextra_X[2]', 
        'MatchedHit_X[3]':'Lextra_X[3]', 
        'MatchedHit_Y[2]':'Lextra_Y[2]',
        'MatchedHit_Y[3]':'Lextra_Y[3]',
        'closest_T_per_station[2]':'MatchedHit_T[2]',
        'closest_T_per_station[3]':'MatchedHit_T[3]'
    }
    if ratio_f == True:
        for station in tqdm([2,3], desc='ratio imputation'): # Assume the ratio of DX(n+1)/DX(n) is the same
            MatchedHit_R = {}
            closesthit_R = {}
            for axis in ['X', 'Y', 'Z']:
                MatchedHit_R[axis] = \
                dataDF['MatchedHit_D{}[{}]'.format(axis,station)].dropna().median() / dataDF['MatchedHit_D{}[{}]'.format(axis,station-1)].dropna().median()
            for axis in ['X', 'Y', 'Z']:
                col = 'MatchedHit_D{}[{}]'.format(axis, station)
                col_prev = 'MatchedHit_D{}[{}]'.format(axis, station-1)
                ind_null = dataDF[col].isnull()
                if ind_null.sum() == 0: 
                    continue
                dataDF.loc[ind_null, col] = dataDF.loc[ind_null, col_prev] * MatchedHit_R[axis]
            if not essential:
                for axis in ['x', 'y']:
                    closesthit_R[axis] = \
                    dataDF['closest_d{}_per_station[{}]'.format(axis,station)].dropna().median() / dataDF['closest_d{}_per_station[{}]'.format(axis,station-1)].dropna().median()
                for axis in ['x', 'y']:
                    col = 'closest_d{}_per_station[{}]'.format(axis,station)
                    col_prev = 'closest_d{}_per_station[{}]'.format(axis,station-1)
                    ind_null = dataDF[col].isnull()
                    if ind_null.sum() == 0: 
                        continue
                    dataDF.loc[ind_null, col] = dataDF.loc[ind_null, col_prev] * closesthit_R[axis]
    
    if substitution_f == True:
        for mcol in tqdm(imputation_dict.keys(), desc="substitution imputation"):
            if mcol not in dataDF.columns:
                continue
            ind_null = dataDF[mcol].isnull()
            if ind_null.sum() == 0: 
                continue
            dataDF.loc[ind_null, mcol] = dataDF.loc[ind_null, imputation_dict[mcol]]
    
    if mean_f == True:
        for col in tqdm(['MatchedHit_Z[2]', 'MatchedHit_Z[3]', 'closest_z_per_station[2]', 'closest_z_per_station[3]',
                         'closest_x_per_station[2]', 'closest_x_per_station[3]', 'closest_y_per_station[2]', 'closest_y_per_station[3]', 
                         'average_x_per_station[2]',  'average_x_per_station[3]',  'average_y_per_station[2]', 'average_y_per_station[3]'], 
                        desc="Mean value imputation"):
            if essential and col in ['average_x_per_station[2]',  'average_x_per_station[3]',  'average_y_per_station[2]', 'average_y_per_station[3]']:
                continue
            if col not in dataDF.columns:
                continue
            ind_null = dataDF[col].isnull()
            if ind_null.sum() == 0: 
                continue
            dataDF.loc[ind_null, col] = dataDF[col].mean()
    
    if angle_f == True:
        for col in tqdm(['MAngle[0]', 'MAngle[1]', 'MAngle', 'MAngle_v2[0]', 'MAngle_v2[1]', 'MAngle_v2[2]'], 
                        desc="angle imputation"):
            if not col in dataDF.columns:
                continue
            ind_null = dataDF[col].isnull()
            if ind_null.sum() == 0: 
                continue
            vec_col = []
            vec_col2 = []
            if col == 'MAngle[0]':
                vec_col = ['MatchedHit_X[0]','MatchedHit_Y[0]','MatchedHit_Z[0]', 
                           'MatchedHit_X[1]', 'MatchedHit_Y[1]', 'MatchedHit_Z[1]']
                vec_col2 =  ['MatchedHit_X[1]','MatchedHit_Y[1]','MatchedHit_Z[1]', 
                           'MatchedHit_X[2]', 'MatchedHit_Y[2]', 'MatchedHit_Z[2]']
            elif col == 'MAngle[1]':
                vec_col =  ['MatchedHit_X[1]','MatchedHit_Y[1]','MatchedHit_Z[1]', 
                           'MatchedHit_X[2]', 'MatchedHit_Y[2]', 'MatchedHit_Z[2]']
                vec_col2 = ['MatchedHit_X[2]','MatchedHit_Y[2]','MatchedHit_Z[2]', 
                           'MatchedHit_X[3]', 'MatchedHit_Y[3]', 'MatchedHit_Z[3]']
            elif col == 'MAngle':
                vec_col = ['MatchedHit_X[0]','MatchedHit_Y[0]','MatchedHit_Z[0]', 
                           'MatchedHit_X[1]', 'MatchedHit_Y[1]', 'MatchedHit_Z[1]']
                vec_col2 = [0, 0, 0, 'MatchedHit_X[0]', 'MatchedHit_Y[0]', 'MatchedHit_Z[0]']
            elif col == 'MAngle_v2[0]':
                vec_col = ['MatchedHit_X[0]','MatchedHit_Y[0]','MatchedHit_Z[0]', 
                           'MatchedHit_X[1]', 'MatchedHit_Y[1]', 'MatchedHit_Z[1]']
                vec_col2 = [0, 0, 0, 'Lextra_X[0]', 'Lextra_Y[0]', 'MatchedHit_Z[0]']
            elif col == 'MAngle_v2[1]':
                vec_col =  ['MatchedHit_X[1]','MatchedHit_Y[1]','MatchedHit_Z[1]', 
                           'MatchedHit_X[2]', 'MatchedHit_Y[2]', 'MatchedHit_Z[2]']
                vec_col2 = [0, 0, 0, 'Lextra_X[1]', 'Lextra_Y[1]', 'MatchedHit_Z[1]']
            elif col == 'MAngle_v2[2]':
                vec_col =  ['MatchedHit_X[2]','MatchedHit_Y[2]','MatchedHit_Z[2]', 
                           'MatchedHit_X[3]', 'MatchedHit_Y[3]', 'MatchedHit_Z[3]']
                vec_col2 = [0, 0, 0, 'Lextra_X[2]', 'Lextra_Y[2]', 'MatchedHit_Z[2]']

            delta = pd.DataFrame(index = dataDF.index[ind_null])
            for i, axis in enumerate(['X', 'Y', 'Z']):
                delta.loc[ind_null, axis+'1'] = dataDF.loc[ind_null, vec_col[3+i]] - dataDF.loc[ind_null, vec_col[i]]
                if col in ['MAngle[0]','MAngle[1]']:
                    delta.loc[ind_null, axis+'2'] = dataDF.loc[ind_null, vec_col2[3+i]] - dataDF.loc[ind_null, vec_col2[i]]
                else:
                    delta.loc[ind_null, axis+'2'] = dataDF.loc[ind_null, vec_col2[3+i]] - vec_col2[i]
            delta = delta[['X1','Y1','Z1','X2','Y2','Z2']]
            dataDF.loc[ind_null, col] = delta.progress_apply(lambda x: angle(x.values), axis=1)
            del delta
    
    if average_xy_f == True:
        for station in tqdm(range(4), desc='average_xy'):
            col = 'closest_xy_per_station[{}]'.format(station)
            if not col in dataDF.columns:
                continue
            ind_null = dataDF[col].isnull()
            if ind_null.sum() == 0:
                continue
            dataDF.loc[ind_null, col] = dataDF.loc[ind_null, 'closest_x_per_station[{}]'.format(station)] + dataDF.loc[ind_null, 'closest_y_per_station[{}]'.format(station)]

    gc.collect()
    return dataDF

def angle(arr):
    x = arr[0:3]
    y = arr[3:]
    dot_xy = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cos = dot_xy / (norm_x*norm_y)
    cos = np.clip(cos, -1, 1)
    rad = np.arccos(cos)
    theta = rad * 180 / np.pi
    return theta

def data_check(dataDF):
    print(dataDF.shape)
    null_col = []
    for col in tqdm(dataDF.columns, desc='checking data...,'):
        ind_null = dataDF[col].isnull()
        if ind_null.sum() == 0:
            continue
        print('{}: {} null items found'.format(col, ind_null.sum()))
        null_col.append(col)
        display(dataDF.loc[ind_null, col].head(5))
    return null_col

### Import preprocessed data from track 1

In [ ]:
trn_data = pd.read_hdf('01_rawdata/trn_154col.hdf') # this is the preprocess train data from track 1

In [ ]:
y_train = trn_data[utils.TRAIN_COLUMNS]

In [ ]:
x_train = trn_data[utils.COL_ESSENTIAL]

In [ ]:
x_train = missing_value_imputation(x_train, essential=True, 
                                   ratio_f=False, mean_f=False, substitution_f=False, angle_f=True, average_xy_f=False)

In [ ]:
_ = data_check(x_train) # some features are left nan deliberately

In [ ]:
del trn_data
gc.collect()

### Something to do with weights

In [ ]:
def weight_flipper(data, threshold=4000, multiplier=2.0):
    ind_negative = (data['weight'] < 0)
    
    ind_label0_negative = (data["weight"] > -threshold) & (data['weight'] < 0) & (data["label"] == 0)
    data.loc[ind_label0_negative, 'weight'] = data['weight'].map(lambda x: multiplier * x)
    
    data.loc[ind_negative, 'weight'] = data['weight'].map(lambda x: -1.0 * x)
    data.loc[ind_negative, 'label'] = data['label'].map(lambda x: 1-x)

    return data

In [ ]:
w_raw = y_train['weight'].copy()
y_raw = y_train['label'].copy()
flipped_y = weight_flipper(y_train)

## Learning

In [ ]:
x_cv_trn, x_cv_test, y_cv_trn, y_cv_test, w_cv_trn, w_cv_test = \
    train_test_split(x_train, flipped_y['label'], flipped_y['weight'], test_size=0.2, shuffle=True, random_state=11)

In [ ]:
x_cv_trn.shape

In [ ]:
train_data = Pool(
    data = x_cv_trn.values,
    label = y_cv_trn.values,
    weight = w_cv_trn.values)
eval_data = Pool(
    data = x_cv_test.values,
    label = y_cv_test.values,
    weight = w_cv_test.values)

In [ ]:
cat_params = {'iterations':10000, 'eval_metric':'RMSE', 'one_hot_max_size':5,
              'use_best_model':True, 'random_state':None, 'thread_count':7}
fit_params = {'early_stopping_rounds':10, 'verbose':False, 'plot':True}

cat = CatBoostRegressor(depth=7, random_strength=40, bagging_temperature=0.2, 
                              learning_rate=0.06, 
                              **cat_params)
cat.fit(X=train_data, eval_set=eval_data, **fit_params)

In [ ]:
importance = pd.DataFrame({'imp': cat.feature_importances_, 'col': x_cv_trn.columns})
_ = importance.plot(kind='barh', x='col', y='imp', figsize=(20, 25))

## Evaluation and model export

In [ ]:
pred = cat.predict(x_cv_test)
scoring.rejection90(y_raw.loc[y_cv_test.index], pred, sample_weight=w_raw.loc[x_cv_test.index].values)

In [ ]:
cat.save_model('cpp_track_2/model.cbm')